In [1]:
import random
import hashlib
from collections import namedtuple

import numpy as np
import numba

In [2]:
Package = namedtuple("Package", ("frame", "y", "x", "payload"))

In [3]:
class FullTile:
    def __init__(self, frame_count, frame_offset):
        self.origin = (frame_offset, 0, 0)
        self.data = np.zeros((frame_count, 2, 2), dtype=np.int64)
        self.damage = np.zeros((frame_count, 2, 2), dtype=np.int8)
        self.unique_count = 0
        self.expected_count = np.prod(self.data.shape, dtype=np.int64)
    
    def complete(self):
        return self.unique_count == self.expected_count

In [4]:
class SingleTile:
    def __init__(self, package, frame_offset):
        self.data = np.array(package.payload, dtype=np.int64).reshape((1, 1))
        self.damage = np.ones((1, 1), dtype=np.int8)
        self.unique_count = 1
        self.expected_count = 1
        self.origin = (frame_offset, package.y, package.x)
        
    def complete(self):
        return True

In [5]:
def read_iter(num_frames=128):
    for f in range(num_frames):
        for y in range(2):
            for x in range(2):
                yield Package(frame=f, y=y, x=x, payload=hash((f, y, x)))

In [6]:
def scramble(packages, window=16):
    buffer = []
    for i in range(window):
        try:
            buffer.append(next(packages))
        except StopIteration:
            break
    while buffer:
        index = random.randint(0, len(buffer) - 1)
        yield buffer.pop(index)
        try:
            buffer.append(next(packages))
        except StopIteration:
            continue

In [7]:
def random_drop(packages, p=0.01):
    for pack in packages:
        if random.random() > p:
            yield pack
            
def random_dup(packages, p=0.01):
    for pack in packages:
        yield pack
        if random.random() <= p:
            yield pack

In [8]:
def block_drop(packages, offset, count):
    counter = 0
    for pack in packages:
        if counter < offset or counter >= (offset + count):
            yield pack
        counter += 1

In [9]:
def straggler(packages, delay, p=0.01, cleanup=True):
    buffer = []
    offsets = set()
    counter = 0
    for pack in packages:
        if random.random() > p:
            yield pack
        else:
#             print("straggle ", pack.frame, pack.y, pack.x)
            buffer.append(pack)
            offsets.add(counter + delay)
        if counter in offsets:
            pack = buffer.pop(0) 
#             print("unstraggle ", pack.frame, pack.y, pack.x)
            yield pack
        counter += 1
    if cleanup:
        for pack in buffer:
    #         print("cleanup ", pack.frame, pack.y, pack.x)
            yield pack

In [10]:
# This emulates a Numba inner loop where we don't want to do buffer management,
# but focus on decoding and writing data to the right place
def numba_inner(packages, tiles_inout, start_frame, stop_frame):
    stragglers = []
    forerunners = []
    processed = 0
    
    def get_frame_index(package, tile):
        return package.frame - tile.origin[0] - start_frame
            
    def decode_into(package, tile_inout, frame_index):
        y = package.y
        x = package.x
        tile_inout.data[frame_index, y, x] = package.payload
        if tile_inout.damage[frame_index, y, x] == 0:
            tile_inout.unique_count += 1
        tile_inout.damage[frame_index, y, x] += 1
    
    for package in packages:
        # Here goes decoding the header
        # ...
        # We drop any packages that are out of range
        if (package.frame < start_frame) or (package.frame >= stop_frame):
            continue
            
        for tile in tiles_inout:
            frame_index = get_frame_index(package, tile)
            if frame_index < 0:
                processed += 1
                stragglers.append(package)
                break
            elif frame_index < tile.data.shape[0]:
                processed += 1
                decode_into(package, tile, frame_index)
                break
        else:  # of for loop
            # We are beyond the last tile
            if frame_index >= tile.data.shape[0]:
                processed += 1
                forerunners.append(package)
                continue        
    return stragglers, forerunners, processed
    

def decode_reorder(packages, frames_per_tile=4, start_frame=0, tile_buffer=3, num_frames=128, yield_stragglers=False):
    tiles = []
    
    tile_end = np.array((0,))
    
    def append_tile(min_forerunner=None):
        # min_forerunner is in dataset coordinates, not package coordinates
        # We may want to bridge a gap
        if min_forerunner is not None:
            # Don't bridge small gaps that fit within a regular tile
            if min_forerunner < tile_end[0] + frames_per_tile:
                start = tile_end[0]
            # Large gap, make a jump
            else:
                start = max(min_forerunner, tile_end)
        else:
            start = tile_end[0]
        length = min(frames_per_tile, num_frames - start)
        if length > 0:
            tiles.append(FullTile(length, start))
            tile_end[:] = start + length
            return True
        else:
            return False
            
    def dispatch(tile):
        # We skip empty tiles
        # Could also yield them empty, depending on application requirements
        # In that case we should also make up a fake tile for gaps
        if tile.unique_count:
            yield tile
            
    def rotate(min_forerunner=None):
        if tiles:
            tile = tiles.pop(0)
            yield from dispatch(tile)
        return append_tile(min_forerunner)
        
    for i in range(tile_buffer):
        append_tile()
        
    stop_frame = start_frame + num_frames
    empty = False
    processed = True
    chunk = []
    n_packages = np.prod(tiles[0].data.shape, dtype=np.int64)
    # Happy case termination: tiles are empty since dataset is filled up and all tiles are rotated out
    # Other termination:
    # * No more packages from source (empty)
    # * No new tile to append and no packages processed in the last round --> yield a last incomplete tile
    # in a timely fashion, don't wait for stragglers until the source runs out, but only until a chunk without
    # relevant packages is reached
    while tiles and not empty and (tile_end < num_frames or processed):
        while len(chunk) < n_packages:
            try:
                chunk.append(next(packages))
            except StopIteration:
                empty = True
                break
        stragglers, forerunners, processed = numba_inner(
            chunk,
            tiles_inout=tiles,
            start_frame=start_frame,
            stop_frame=stop_frame
        )
        # The packages are completely handled
        chunk = []
        if stragglers and yield_stragglers:
            for straggler in stragglers:
                yield SingleTile(straggler, straggler.frame - start_frame)
        # We catch up with forerunners by rotating in a new tile with an offset
        # Since packets can't time-travel, forerunners likely mean that we had a lot of packets drop,
        # a larger gap
        if forerunners:
            min_forerunner = min(p.frame for p in forerunners)
            moved = yield from rotate(min_forerunner - start_frame)
            # Forerunners while no new tile to append is a bug
            assert moved
            # We process the forerunners again in the next rotation
            empty = False
            chunk = forerunners
        while tiles and (tiles[0].complete() or ((tile_end < num_frames) and tiles[-1].unique_count)):
            yield from rotate()
#         print(len(tiles), empty, tile_end < num_frames, processed)
    for tile in tiles:
        yield from dispatch(tile)
    

In [11]:
start_frame = 37
num_frames = 678

account = {}

def account_for(packages):
    for package in packages:
        account[(package.frame, package.y, package.x)] = package.payload
        yield package

source = account_for(straggler(
    block_drop(
        random_dup(
            random_drop(
                scramble(
                    read_iter(1024),
                    window=4
                ),
                p=0.1
            ),
            p=0.01,
        ),
        offset=273,
        count=160,
    ),
    delay=10,
    p=0.1,
    cleanup=False
))

# source = account_for(random_drop(
#     scramble(
#         read_iter(1024),
#         window=4,
#     ),
#     p=0.1,
# ))

# source = account_for(block_drop(
#     read_iter(1024),
#     offset=600,
#     count=200,
# ))


# source = account_for(read_iter(1024))

for tile in decode_reorder(source, start_frame=start_frame, num_frames=num_frames, yield_stragglers=True):
    print(type(tile).__name__, tile.origin, tile.data.shape, tile.complete(), np.where(tile.damage < 1))
    if isinstance(tile, FullTile):
        for frame in range(tile.data.shape[0]):
            for y in range(2):
                for x in range(2):
                    key = (frame + tile.origin[0] + start_frame, y, x)
                    expected = hash(key)
                    if tile.damage[frame, y, x]:
                        if tile.data[frame, y, x] == expected:
                            account[key] = True
                        else:
                            print(f"misordered full {frame} {y} {x}")
    elif isinstance(tile, SingleTile):
        frame, y, x = tile.origin
        key = (frame + start_frame, y, x)
        expected = hash(key)
        
        if tile.data == expected:
            account[key] = True
        else:
            print(f"misordered single {frame} {y} {x}")

remaining = 0
for item in source:
    remaining += 1
print("remaining: ", remaining)
            
for key, value in account.items():
    frame, y, x = key
    if frame >= start_frame and frame < (start_frame + num_frames) and value is not True:
        print(f"lost: {key} ({key[0] - start_frame})" )

FullTile (0, 0, 0) (4, 2, 2) False (array([0, 2], dtype=int64), array([0, 1], dtype=int64), array([0, 1], dtype=int64))
FullTile (4, 0, 0) (4, 2, 2) False (array([2], dtype=int64), array([1], dtype=int64), array([0], dtype=int64))
FullTile (8, 0, 0) (4, 2, 2) False (array([1, 3, 3], dtype=int64), array([0, 0, 1], dtype=int64), array([1, 0, 0], dtype=int64))
FullTile (12, 0, 0) (4, 2, 2) False (array([3], dtype=int64), array([1], dtype=int64), array([1], dtype=int64))
FullTile (16, 0, 0) (4, 2, 2) False (array([0, 1, 3], dtype=int64), array([1, 1, 1], dtype=int64), array([1, 0, 1], dtype=int64))
FullTile (20, 0, 0) (4, 2, 2) False (array([0, 1, 3, 3], dtype=int64), array([1, 0, 0, 0], dtype=int64), array([1, 1, 0, 1], dtype=int64))
FullTile (24, 0, 0) (4, 2, 2) True (array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
FullTile (28, 0, 0) (4, 2, 2) False (array([3], dtype=int64), array([1], dtype=int64), array([1], dtype=int64))
FullTile (32, 0, 0) (4, 2, 2) False (ar

In [12]:
@numba.njit
def test(t):
    print(t.frame)

In [13]:
test(Package(frame=0, y=0, x=0, payload=1))

0
